In [1]:
import pandas as pd
import csv
import gzip
import json

def read_pricing_data(input_filename):
    rows = []
    with gzip.open(input_filename, 'rt') as f:
        try:
            for line in f:
                row = json.loads(line.rstrip("\n"))
                #print(row)
                timestamp_ms_received = row[0]
                price = row[2]
                rows.append({'timestamp_ms_received': timestamp_ms_received, 'price': price})
        except EOFError:
            pass

    df = pd.DataFrame(rows)
    df["timestamp"] = pd.to_datetime(df["timestamp_ms_received"], unit="ms")
    df.set_index('timestamp', inplace=True)

    return df


In [21]:
def process_pricing_data(df,output_filename):
    df = df.resample('1S').last()
    rolling_std = df['price'].rolling(window=10, min_periods=1).std()
    result = pd.DataFrame({
        'timestamp': df.index,
        'std': rolling_std.values
    })
    result['timestamp'] = result['timestamp'].astype(int) // 10**6
    result.to_csv(output_filename, index=False)
    return result

In [22]:
price_df = read_pricing_data('tf-binance-BTC_ETH-2022-07-21.gz')
process_pricing_data(price_df,'out.csv')

,timestamp,std
0,1658361600000,NaN
1,1658361601000,0.000000
2,1658361602000,0.000012
3,1658361603000,0.000011
4,1658361604000,0.000011
...,...,...
32789,1658394389000,0.000028
32790,1658394390000,0.000034
32791,1658394391000,0.000039
32792,1658394392000,0.000042
